In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from statsmodels.tsa.arima.model import ARIMA
from jupyter_dash import JupyterDash 
import global_variables


data = pd.read_excel('excelformat.xlsx')
#data["Debit"]=data["Debit"].replace("-","")
#data = data.drop(['ValueDate','Description','Ref No./ChequeNo','Credit','Balance'], axis=1)
data["Debit"]=data["Debit"].fillna(0)
#data["Debit"]=pd.to_numeric(data["Debit"])
data["Debit"] = [float(str(i).replace(",", "")) 
for i in data["Debit"]]
df = pd.DataFrame(data, columns=['Txn Date','Debit'])
df = df.rename(columns={'Txn Date': 'Date'})



# Clean and preprocess the data
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')
df = df.resample('D').sum()



actual_data = pd.read_excel('actualvalues.xlsx')
actual_data = actual_data.drop(['Value','Description','Cheque','Credit','Balance'], axis=1)
actual_data["Debit"]=actual_data["Debit"].fillna(0)
#data["Debit"]=pd.to_numeric(data["Debit"])
actual_data["Debit"] = [float(str(i).replace(",", "")) 
for i in actual_data["Debit"]]
actual_df = pd.DataFrame(actual_data, columns=['Txn Date','Debit'])
actual_df = actual_df.rename(columns={'Txn Date': 'Date'})
actual_df['Date'] = pd.to_datetime(actual_df['Date'])
actual_df = actual_df.set_index('Date')
actual_df =actual_df.resample('D').sum()






# Fit an ARIMA model to the data
model = ARIMA(df['Debit'], order=(2, 1, 2))
model_fit = model.fit()

# Make predictions for the next 30 days
forecast_values = model_fit.forecast(steps=30)

# Create a Dash app
app = JupyterDash(__name__)

# Define the layout of the app
app.layout = html.Div(children=[
    html.H1(children='PREDICT EXPENSES FOR NEXT 30 DAYS',
        style = {"font-size": 35,
        'color': '#ffff','text-align':'center'}, ),

    

    dcc.Graph(
        id='debit-forecast-graph',
        figure={
            'data': [
                go.Scatter(
                    x=df.index,
                    y=df['Debit'],
                    name='Actual'
                ),
                go.Scatter(
                    x=forecast_values.index,
                    y=forecast_values.values,
                    name='Forecast'
                )
            ],
            'layout': {
                'title': 'Debit Expense Forecasting',
                'xaxis': {
                    'title': 'Date'
                },
                'yaxis': {
                    'title': 'Debit Expense Amount'
                }
            }
        }
    ),
     dcc.Graph(
        id='debit-graph',
        figure={
            'data': [
                {'x': df.index, 'y': df['Debit'], 'type': 'line', 'name': 'Actual'},
                {'x': actual_df.index, 'y': actual_df['Debit'], 'type': 'line', 'name': 'Actual Next 30 Days'},
                {'x': forecast_values.index, 'y': forecast_values.values, 'type': 'line', 'name': 'Predicted Next 30 Days'},
            ],
            'layout': {
                'title': 'Debit Data'
            }
        }
    )
])

if __name__ == '__main__':
    app.run_server(port=8887)


C:\Users\Nandhini\AppData\Local\Temp\ipykernel_20024\3984657101.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Nandhini\AppData\Local\Temp\ipykernel_20024\3984657101.py:6: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


               Debit
Date                
2020-09-01     99.00
2020-09-02      0.00
2020-09-03   1425.55
2020-09-04      0.00
2020-09-05   1152.00
2020-09-06      0.00
2020-09-07  15000.00
2020-09-08    300.00
2020-09-09   2788.00
2020-09-10   5399.00
2020-09-11      0.00
2020-09-12      0.00
2020-09-13   1200.00
2020-09-14    400.00
2020-09-15   1525.00
2020-09-16   9199.00
2020-09-17      0.00
2020-09-18    100.00
2020-09-19  11500.00
2020-09-20      0.00
2020-09-21    129.00
2020-09-22    599.00
2020-09-23   2000.00
2020-09-24      0.00
2020-09-25    374.00
Dash is running on http://127.0.0.1:8887/

Dash app running on http://127.0.0.1:8887/
